### Setup
---

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1. Import data
---

In [10]:
filename = 'pricing_per_service_data.xlsx'
data = pd.read_excel(filename)
print(data)

        Services   Pricing
0      Service 2    241.46
1      Service 1    332.71
2      Service 5  42793.33
3      Service 1    441.33
4      Service 2    274.51
...          ...       ...
77433  Service 1    504.67
77434  Service 1    237.33
77435  Service 5      0.67
77436  Service 3    475.33
77437  Service 1    237.33

[77438 rows x 2 columns]
